In [69]:
import pandas as pd
import os, re
from helpers import *
import datetime

In [70]:
############
# input VARS

PERSON = "taylor"
TRYS = 1
VALID_LABELS = ["lights-on", "turn-off"]
NUM_CHANNELS = 22
TODAY = False

############
# const VARS

DATE = datetime.datetime.today().strftime('%m_%d')
if TODAY == True:
    FNAME = "{}_{}_{}".format(PERSON,DATE,TRYS)
else:
    FNAME = "taylor_12_06_01"
ROOT = os.getcwd() + "/images_scaled/" + FNAME
OUTPUT = os.getcwd() + "/train_csv/" + FNAME + ".csv"
DATE_PATTERN = "[0-9]{2}_[0-9]{2}"
IMG_EXT = ".png"
VERBOSITY = 100
CATS, MONTHS, DAYS, LABELS, SEQ, SETS = [], [], [], [], [], []
for i in range(1, NUM_CHANNELS+1):
    globals()["PATH{}".format(i)] = []
    
print(ROOT)

/Users/kyy/cerebro_train/images_scaled/taylor_12_06_01


In [71]:
def make_df_from_images(image_root):
    for cat in [d for d in os.listdir(image_root) if "voice" in d]:
        cat_path = os.path.join(image_root, cat)
        for date in [d for d in os.listdir(cat_path) if re.match(DATE_PATTERN, d)]:
            print("\tProcessing {}".format(date))
            date_path = os.path.join(cat_path, date)
            month = int(date[:2])
            day = int(date[3:])
            date_count = 0
            for label in [d for d in os.listdir(date_path) if d in VALID_LABELS]:
                label_path = os.path.join(date_path, label)
                placeholder = os.path.join(label_path, "ch1")
                for image in [f for f in os.listdir(placeholder) if f.endswith(IMG_EXT)]:
                    date_count += 1
                    for i in range(1, NUM_CHANNELS+1):
                        p = os.path.join(label_path, "ch{}".format(i), image)
                        if os.path.exists(p):
                            globals()["PATH{}".format(i)].append(p)
                        else:
                            globals()["PATH{}".format(i)].append(float('nan'))
                    CATS.append(cat)
                    DAYS.append(day)
                    MONTHS.append(month)
                    LABELS.append(label)
                    sequence_number = int(image[:-4])
                    basenum = sequence_number % 10
                    SEQ.append(sequence_number)
                    if basenum < 3:
                        SETS.append("Training")
                    elif basenum < 4:
                        SETS.append("Validation")
                    else:
                        SETS.append("Testing")
            print("\t\tProcessed {} sequences".format(date_count))
    d = {
            "Category":CATS,
            "Day":DAYS,
            "Month":MONTHS,
            "Label":LABELS,
            "SequenceNumber":SEQ,
            "Set":SETS
        }
    for i in range(1, NUM_CHANNELS+1):
        d["Path{}".format(i)] = globals()["PATH{}".format(i)]
    return pd.DataFrame(d)

In [72]:
####################################
# turn spectrogram paths to csv file

df = timer(make_df_from_images, ROOT)
df.to_csv(OUTPUT, index=False)

Start: 2018-12-07 12:49:05.867372
	Processing 12_06
		Processed 10115 sequences
End: 2018-12-07 12:49:08.317879
Finished in 00:00:00:02.45


In [73]:
print(set(df['Set']))
from collections import Counter
c = Counter(df['Set'])

print( c.items() )

{'Testing', 'Training', 'Validation'}
dict_items([('Training', 6069), ('Validation', 2023), ('Testing', 2023)])


In [74]:
print(df.tail(5))

       Category  Day  Month     Label  SequenceNumber         Set  \
10110  no_voice    6     12  turn-off            5580    Training   
10111  no_voice    6     12  turn-off            9392    Training   
10112  no_voice    6     12  turn-off            8932    Training   
10113  no_voice    6     12  turn-off            3873  Validation   
10114  no_voice    6     12  turn-off            1902    Training   

                                                   Path1  \
10110  /Users/kyy/cerebro_train/images_scaled/taylor_...   
10111  /Users/kyy/cerebro_train/images_scaled/taylor_...   
10112  /Users/kyy/cerebro_train/images_scaled/taylor_...   
10113  /Users/kyy/cerebro_train/images_scaled/taylor_...   
10114  /Users/kyy/cerebro_train/images_scaled/taylor_...   

                                                   Path2  \
10110  /Users/kyy/cerebro_train/images_scaled/taylor_...   
10111  /Users/kyy/cerebro_train/images_scaled/taylor_...   
10112  /Users/kyy/cerebro_train/images_scale

In [75]:
print(df.describe())

           Day    Month  SequenceNumber  Path5  Path6  Path7  Path8  Path9  \
count  10115.0  10115.0    10115.000000    0.0    0.0    0.0    0.0    0.0   
mean       6.0     12.0     5054.501236    NaN    NaN    NaN    NaN    NaN   
std        0.0      0.0     2920.094391    NaN    NaN    NaN    NaN    NaN   
min        6.0     12.0        0.000000    NaN    NaN    NaN    NaN    NaN   
25%        6.0     12.0     2524.000000    NaN    NaN    NaN    NaN    NaN   
50%        6.0     12.0     5053.000000    NaN    NaN    NaN    NaN    NaN   
75%        6.0     12.0     7582.500000    NaN    NaN    NaN    NaN    NaN   
max        6.0     12.0    10114.000000    NaN    NaN    NaN    NaN    NaN   

       Path10  Path11   ...    Path13  Path14  Path15  Path16  Path17  Path18  \
count     0.0     0.0   ...       0.0     0.0     0.0     0.0     0.0     0.0   
mean      NaN     NaN   ...       NaN     NaN     NaN     NaN     NaN     NaN   
std       NaN     NaN   ...       NaN     NaN     NaN 

In [76]:
df

,Category,Day,Month,Label,SequenceNumber,Set,Path1,Path2,Path3,Path4,...,Path13,Path14,Path15,Path16,Path17,Path18,Path19,Path20,Path21,Path22
0,no_voice,6,12,lights-on,8081,Training,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,no_voice,6,12,lights-on,4863,Validation,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,no_voice,6,12,lights-on,6912,Training,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,no_voice,6,12,lights-on,6090,Training,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,no_voice,6,12,lights-on,8903,Validation,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,no_voice,6,12,lights-on,2590,Training,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,no_voice,6,12,lights-on,3842,Training,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,no_voice,6,12,lights-on,2584,Testing,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,no_voice,6,12,lights-on,393,Validation,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,no_voice,6,12,lights-on,1933,Validation,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,/Users/kyy/cerebro_train/images_scaled/taylor_...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
